In [1]:
# Evaluation of experiments
import matplotlib.pyplot as plt
import pandas as pd

dfExperiments45 = pd.read_csv('experiments_similarityScore_45_negativeSamples_3_2019-01-07T15-19-40.063.csv', sep=',', low_memory=False, header=None)
dfExperiments45.columns = ['Timestamp', 'Experiment', 'Class', 'TargetTable', 'Precision', 'Recall', 'F-1', 'Blocker', 'BlockingFunction', 'TrainedOn', 'TrainedModel']
dfExperiments45['Indexing Threshold'] = '0.45'

dfExperiments70 = pd.read_csv('experiments_similarityScore_70_negativeSamples_3_2019-01-07T14-39-25.415.csv', sep=',', low_memory=False, header=None)
dfExperiments70.columns = ['Timestamp', 'Experiment', 'Class', 'TargetTable', 'Precision', 'Recall', 'F-1', 'Blocker', 'BlockingFunction', 'TrainedOn', 'TrainedModel']
dfExperiments70['Indexing Threshold'] = '0.7'

In [6]:
#Keep only baselines
dfExperiments45 = dfExperiments45[~dfExperiments45['Experiment'].str.contains('transfer_')]
dfExperiments70 = dfExperiments70[~dfExperiments70['Experiment'].str.contains('transfer_')]
#display(dfExperiments)

#Compare baselines to matching rule
def compareLearnedRuleWithBaseline(dfExperiments):

    dfBaselineExperimentsCompared = pd.DataFrame()

    for targetTable in dfExperiments['TargetTable'].unique():
        dfSubSetExperiments = dfExperiments[dfExperiments['TargetTable'] == targetTable]
        dfLearnedMatchingRule = dfSubSetExperiments[dfSubSetExperiments['Experiment'].str.contains('learned_')]
        dfBaselinesMatchingRule = dfSubSetExperiments[dfSubSetExperiments['Experiment'].str.contains('baseline_')]
    
        for indexLearned, rowLearned in dfLearnedMatchingRule.iterrows():
            dfBaselineExperimentsTmpCompared = pd.DataFrame()
            for indexBaseline, rowBaseline in dfBaselinesMatchingRule.iterrows():
                newRow = rowLearned.copy()
                newRow['Precision-Delta'] = rowLearned['Precision'] - rowBaseline['Precision']
                newRow['Recall-Delta'] = rowLearned['Recall'] - rowBaseline['Recall']
                newRow['F-1-Delta'] = rowLearned['F-1'] - rowBaseline['F-1']
                newRow['Experiment'] = rowBaseline['Experiment']
            
                dfBaselineExperimentsTmpCompared = dfBaselineExperimentsTmpCompared.append(newRow, ignore_index=True)
            
            dfBaselineExperimentsTmpCompared = dfBaselineExperimentsTmpCompared.sort_values(by=['F-1-Delta']).iloc[0]
            dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.append(dfBaselineExperimentsTmpCompared)
    return dfBaselineExperimentsCompared

dfBaselineExperimentsCompared = compareLearnedRuleWithBaseline(dfExperiments45)
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.append(compareLearnedRuleWithBaseline(dfExperiments70))
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.sort_values(by=['TargetTable'])

dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.filter(items=['TargetTable', 'Indexing Threshold',  'TrainedModel', 'Precision', 'Recall',  'F-1', 'Experiment','Precision-Delta',  'Recall-Delta','F-1-Delta'])
dfBaselineExperimentsCompared['Experiment'] = dfBaselineExperimentsCompared['Experiment'].str.replace('baseline_', '').str.replace('_author', '')
dfBaselineExperimentsCompared['Experiment'] = dfBaselineExperimentsCompared['Experiment'].str.split('-', expand=True)[0]
dfBaselineExperimentsCompared['TrainedModel'] = dfBaselineExperimentsCompared['TrainedModel'].str.replace('J48', 'DT')
dfBaselineExperimentsCompared['TrainedModel'] = dfBaselineExperimentsCompared['TrainedModel'].str.replace('SimpleLogistic', 'LR')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('DnbDataAuthors.csv', 'DBpedia-DNB')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('VIAFDataAuthors.csv', 'DBpedia-VIAF')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('WikiDataAuthors.csv', 'DBpedia-Wikidata')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('all', 'DBpedia-All')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('bag_of_words', 'Bag-of-Words')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('label', 'Label')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.round(3)
display(dfBaselineExperimentsCompared)

print(dfBaselineExperimentsCompared.to_latex(index=False))

,TargetTable,Indexing Threshold,TrainedModel,Precision,Recall,F-1,Experiment,Precision-Delta,Recall-Delta,F-1-Delta
1,DBpedia-DNB,0.45,DT,0.896,0.968,0.930,Bag-of-Words,0.133,-0.016,0.071
1,DBpedia-DNB,0.45,LR,0.824,0.984,0.897,Bag-of-Words,0.062,0.000,0.038
1,DBpedia-DNB,0.7,DT,0.967,0.935,0.951,Bag-of-Words,0.204,-0.048,0.092
1,DBpedia-DNB,0.7,LR,0.924,0.984,0.953,Bag-of-Words,0.162,0.000,0.094
1,DBpedia-VIAF,0.45,DT,0.676,0.990,0.803,Bag-of-Words,-0.178,0.222,-0.005
1,DBpedia-VIAF,0.45,LR,0.742,0.990,0.848,Bag-of-Words,-0.112,0.222,0.040
1,DBpedia-VIAF,0.7,DT,0.970,0.980,0.975,Bag-of-Words,0.116,0.212,0.166
1,DBpedia-VIAF,0.7,LR,0.970,0.980,0.975,Bag-of-Words,0.116,0.212,0.166
0,DBpedia-Wikidata,0.45,DT,0.698,1.000,0.822,Label,0.028,0.020,0.026
0,DBpedia-Wikidata,0.45,LR,0.710,1.000,0.830,Label,0.040,0.020,0.034


\begin{tabular}{lllrrrlrrr}
\toprule
      TargetTable & Indexing Threshold & TrainedModel &  Precision &  Recall &    F-1 &    Experiment &  Precision-Delta &  Recall-Delta &  F-1-Delta \\
\midrule
      DBpedia-DNB &               0.45 &           DT &      0.896 &   0.968 &  0.930 &  Bag-of-Words &            0.133 &        -0.016 &      0.071 \\
      DBpedia-DNB &               0.45 &           LR &      0.824 &   0.984 &  0.897 &  Bag-of-Words &            0.062 &         0.000 &      0.038 \\
      DBpedia-DNB &                0.7 &           DT &      0.967 &   0.935 &  0.951 &  Bag-of-Words &            0.204 &        -0.048 &      0.092 \\
      DBpedia-DNB &                0.7 &           LR &      0.924 &   0.984 &  0.953 &  Bag-of-Words &            0.162 &         0.000 &      0.094 \\
     DBpedia-VIAF &               0.45 &           DT &      0.676 &   0.990 &  0.803 &  Bag-of-Words &           -0.178 &         0.222 &     -0.005 \\
     DBpedia-VIAF &               0.

In [11]:
import csv

dfExperimentsAll = dfBaselineExperimentsCompared[dfBaselineExperimentsCompared['TargetTable'].str.contains('All')]
dfExperimentsOther = dfBaselineExperimentsCompared[~dfBaselineExperimentsCompared['TargetTable'].str.contains('All')]

dfAggregatedExperiments = dfExperimentsOther.groupby(['Indexing Threshold']).mean()
dfAggregatedExperiments['Level'] = 'Other'
dfAggregatedExperimentsAll = dfExperimentsAll.groupby(['Indexing Threshold']).mean()
dfAggregatedExperimentsAll['Level'] = 'All'

dfAggregatedExperiments = dfAggregatedExperiments.append(dfAggregatedExperimentsAll)
display(dfAggregatedExperiments)
dfAggregatedExperiments.to_csv('rq1_aggregated_results_author.csv', sep=',', encoding='utf-8', quotechar='"', quoting=csv.QUOTE_ALL)

,Precision,Recall,F-1,Precision-Delta,Recall-Delta,F-1-Delta,Level
Indexing Threshold,,,,,,,
0.45,0.757667,0.988667,0.855000,-0.004500,0.0780,0.034,Other
0.7,0.957000,0.976500,0.966333,0.194833,0.0660,0.145,Other
0.45,0.727500,0.986500,0.837000,0.051500,0.0675,0.058,All
0.7,0.953000,0.975500,0.964000,0.277000,0.0565,0.185,All


In [ ]:
#Rearrange DataFrame
def rearrange(df):
    dfAllExperimentsRearranged = pd.DataFrame()
    for index, row in df.iterrows():
        rowPrecision = row
        rowPrecision['ResultValue'] = row['Precision-Delta']
        rowPrecision['ResultValueType'] = 'Precision-Delta'
        
        dfAllExperimentsRearranged = dfAllExperimentsRearranged.append(rowPrecision)
        
        rowRecall = row
        rowRecall['ResultValue'] = row['Recall-Delta']
        rowRecall['ResultValueType'] = 'Recall-Delta'
        
        dfAllExperimentsRearranged = dfAllExperimentsRearranged.append(rowPrecision)
        
        #rowF1 = row
        #rowF1['ResultValue'] = row['F-1']
        #rowF1['ResultValueType'] = 'F-1'
        
        #dfAllExperimentsRearranged = dfAllExperimentsRearranged.append(rowF1)
    
    return dfAllExperimentsRearranged


dfAllExperimentsRearranged = rearrange(dfBaselineExperimentsCompared)

display(dfAllExperimentsRearranged)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plotResults(df, title):
    sns.set(style="whitegrid")
    fig, ax1 = plt.subplots(figsize=(20,8))
    sns.catplot(x="shortExperiment", y="ResultValue", hue="ResultValueType", data=df, height=6, kind="bar", ax=ax1)
    
    titleAll = title
    ax1.set_title(titleAll, fontweight="bold", size=20) # Title
    ax1.set_xlabel("Experiments",fontsize = 20.0)
    ax1.set_ylabel("" ,fontsize = 20.0) # Y label
    ax1.tick_params(direction='out', length=6, width=2, labelsize=20, grid_alpha=0.5)
    ax1.tick_params(axis = 'x', labelrotation=60)
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=20)


dfAllExperimentsRearranged['shortExperiment'] = dfAllExperimentsRearranged['TrainedModel'].str.cat(dfAllExperimentsRearranged['Experiment'].str.replace('Authors', '').str.replace('_author', '').str.replace('baseline_', '').str.replace('.csv',''), sep='-')
plotResults(dfAllExperimentsRearranged, 'Baseline Comparison')

In [ ]:
display(dfAllExperimentsRearranged.filter(items=['shortExperiment','Precision', 'Recall', 'F-1']).drop_duplicates(keep='first'))

In [ ]:
round(dfAllExperimentsRearranged.filter(items=['shortExperiment', 'F-1-Delta']).drop_duplicates(keep='first')['F-1-Delta'].mean(),4)